In [1]:
import os
import graphviz
from langchain_groq import ChatGroq
from langchain_core.tools import tool
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
from langchain.agents import Tool, initialize_agent
from langgraph.graph import StateGraph, END
from typing import TypedDict

In [3]:
os.environ["GROQ_API_KEY"] = "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"

Tools

In [5]:
# Agent 1 Tools
def extract_mood(query: str) -> str:
    """Simulate mood extraction based on query"""
    if "adventurous" in query:
        return "adventurous"
    elif "relaxed" in query:
        return "relaxed"
    return "neutral"

def decide_destination(mood: str) -> str:
    """Simulate holiday destination selection based on mood"""
    if mood == "adventurous":
        return "Mountains"
    elif mood == "relaxed":
        return "Beach"
    return "City"

# Agent 2 Tools
def extract_budget(query: str) -> float:
    """Simulate budget extraction from the query"""
    if "budget of" in query:
        budget = float(query.split("budget of $")[1].split()[0])
        return budget
    return 1000  # Default budget if none is provided

def book_flight(budget: float) -> str:
    """Simulate flight booking based on the budget"""
    if budget >= 1500:
        return "Business class flight booked."
    return "Economy class flight booked."

# Agent 3 Tools
def check_room_availability(destination: str) -> bool:
    """Simulate room availability based on the destination"""
    # For simplicity, assume that rooms are always available at the destination
    return True

def book_hotel(destination: str) -> str:
    """Simulate hotel booking based on destination"""
    if destination == "Mountains":
        return "Booked a resort with adventure activities."
    elif destination == "Beach":
        return "Booked a beachfront hotel."
    return "Booked a city hotel."

# Agent 4 Tools
def generate_trip_summary(flight: str, hotel: str) -> str:
    """Generate a trip summary"""
    return f"Trip Summary: {flight} and {hotel}."


Agents

In [7]:
# Define Agent 1: Extract Mood and Decide Destination
def agent_1(state):
    user_input = state["input"]
    mood = extract_mood(user_input)  # Extract mood from the input
    destination = decide_destination(mood)  # Decide destination based on mood
    return {"input": user_input, "mood": mood, "destination": destination}

# Define Agent 2: Extract Budget and Book Flight
def agent_2(state):
    user_input = state["input"]
    destination = state["destination"]
    budget = extract_budget(user_input)  # Extract budget from the input
    flight = book_flight(budget)  # Book flight based on budget
    return {"flight": flight,"destination": destination}

# Define Agent 3: Book Hotel Based on Destination
def agent_3(state):
    destination = state["destination"]
    flight = state["flight"]
    if check_room_availability(destination):  # Check if rooms are available
        hotel = book_hotel(destination)  # Book hotel based on destination
        return {"hotel": hotel,"flight": flight}
    return {"hotel": "No rooms available.","flight": flight}

# Define Agent 4: Provide a Summary of the Trip
def agent_4(state):
    flight = state["flight"]
    hotel = state["hotel"]
    summary = generate_trip_summary(flight, hotel)  # Generate trip summary
    return {"summary": summary}


In [8]:
def router_node(state):
    # Returning a dictionary that signals the end of the flow
    return state

In [9]:
graph = StateGraph(dict)

In [10]:
graph.add_node("agent_1", agent_1)       # Extract mood → decide destination
graph.add_node("agent_2", agent_2)       # Extract budget → book flight
graph.add_node("agent_3", agent_3)       # Book hotel
graph.add_node("agent_4", agent_4)       # Generate summary
graph.add_node("router", router_node)    # Optional routing or final cleanup

graph.set_entry_point("agent_1")

# Connect nodes properly
graph.add_edge("agent_1", "agent_2")     # Mood and destination → agent_2
graph.add_edge("agent_2", "agent_3")     # Budget and flight → agent_3
graph.add_edge("agent_3", "agent_4")     # Hotel → agent_4
graph.add_edge("agent_4", "router")      # Trip summary → router
graph.add_edge("router", END)


In [11]:
app = graph.compile()

In [12]:
# Sample User Query
user_input = "I am feeling adventurous and have a budget of $1200 for my trip."

In [13]:
# Run the LangGraph with the user query
initial_state = {"input": user_input}

In [14]:
output = app.invoke({"input": user_input})

In [31]:
output["summary"]

'Trip Summary: Economy class flight booked. and Booked a resort with adventure activities..'

In [16]:










# Output the final summary from Agent 4
print(output["summary"])  # Print the trip summary from Agent 4


Trip Summary: Economy class flight booked. and Booked a resort with adventure activities..
